In [3]:
def create_sbatches(EXP_NUMBER, INPUT_DIM, BSIZE, LR_RATE, STEP, GAMMA):
    STRING = f'#!/bin/sh                         \n' \
             '#SBATCH -N 1                       \n' \
             '#SBATCH --job-name=MAD_RANDOM      \n' \
             '#SBATCH --time=5:59:00             \n' \
             '#SBATCH --cpus-per-task=6          \n' \
             '#SBATCH --mem=180GB                \n' \
             '#SBATCH --gres=gpu:2               \n' \
             '#SBATCH --chdir=/ibex/ai/home/soldanm/projects/nips22/episodic-memory/MQ/    \n' \
             '#SBATCH -o logs/%x_%A_%a.out       \n' \
             '#SBATCH -e logs/%x_%A_%a.err       \n' \
             '#SBATCH --array=0                  \n' \
             '                                   \n' \
             'module load gcc                    \n' \
             '                                   \n' \
             'echo "######################### SLURM JOB ########################"          \n' \
             'echo HOST NAME                     \n' \
             'echo `hostname`                    \n' \
             'echo "############################################################"          \n' \
             '                                   \n' \
             'environment=ego                    \n' \
             'conda_root=$HOME/anaconda3         \n' \
             'source $conda_root/etc/profile.d/conda.sh                                     \n' \
             'conda activate $environment        \n' \
             'set -ex                            \n' \
             '# ------------------------ need not change -----------------------------------                              \n' \
             'FEATURES=/ibex/ai/home/soldanm/projects/nips22/episodic-memory/MQ/Evaluation/ego4d/egoclip_egonce            \n' \
             f'CHECKPOINT=./outputs/hps_search_egoclip_egonce_features/{EXP_NUMBER}/                                       \n' \
             f'INPUT_DIM={INPUT_DIM}             \n' \
             f'BSIZE={BSIZE}                     \n' \
             f'LR_RATE={LR_RATE}                 \n' \
             f'STEP={STEP}                       \n' \
             f'GAMMA={GAMMA}                     \n' \
             '                                   \n' \
             'python Train.py --use_xGPN --is_train true --dataset ego4d --feature_path $FEATURES --checkpoint_path $CHECKPOINT --batch_size $BSIZE --train_lr $LR_RATE --step_size $STEP --step_gamma $GAMMA --input_feat_dim $INPUT_DIM                  \n' \
             'python Infer.py --use_xGPN --is_train false --dataset ego4d --feature_path $FEATURES --checkpoint_path $CHECKPOINT --input_feat_dim $INPUT_DIM --output_path $CHECKPOINT                  \n' \
             'python Eval.py --dataset ego4d --output_path $CHECKPOINT --out_prop_map true --eval_stage all > ${CHECKPOINT}/results.txt ' 
    
    text_file = open(f"./generated/HPS_SEARCH_{EXP_NUMBER}.sh", "w")
    n = text_file.write(STRING)
    text_file.close()

In [4]:
EXP_NUMBER = 0
INPUT_DIM=256

for BSIZE in [8,16,32]:
    for LR_RATE in [0.0005,0.0001,0.00005,0.00001]:
        for STEP in [5,15,30]:
            for GAMMA in [0.5,0.25,0.1,0.05]:
                create_sbatches(EXP_NUMBER, INPUT_DIM, BSIZE, LR_RATE, STEP, GAMMA)
                EXP_NUMBER+=1
                        
print(f'Total Number of experiments {EXP_NUMBER}.')


Total Number of experiments 144.
